In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import copy
import os
import math

In [4]:
# import the differet datafiles from the dat_mod folder
csv_files = os.listdir("Data/lsms/mod_data/")
dfs  = []
for file in csv_files:
    path = "Data/lsms/mod_data/" + file
    dfs.append(pd.read_csv(path,index_col = 0))
geos = pd.concat(dfs).reset_index(drop = True)
geos['year_start'] = [i[:4] for i in list(geos['year'])]
geos['year_end'] = [i[-4:] for i in list(geos['year'])]
geos['clusterid'] = [i[0]+"_"+str(i[1]) for idx,i in geos[['country','clusterid']].iterrows()]
geos['cluster_year_id'] = [i[0]+"_"+str(i[1]) for idx, i in geos[['clusterid','round']].iterrows()]

,country,clusterid,lat,lon,round,year,n_households,year_start,year_end,cluster_year_id
0,ug,ug_10210001,0.333188,32.570656,3,2011/2012,4,2011,2012,ug_10210001_3
1,ug,ug_10210001,0.333188,32.570656,4,2013/2014,3,2013,2014,ug_10210001_4
2,ug,ug_10210001,0.333188,32.570656,5,2015/2016,3,2015,2016,ug_10210001_5
3,ug,ug_10210001,0.333188,32.570656,2,2010/2011,5,2010,2011,ug_10210001_2
4,ug,ug_10210001,0.333188,32.570656,1,2009/2010,6,2009,2010,ug_10210001_1
...,...,...,...,...,...,...,...,...,...,...
5938,tz,tz_55218021,-5.367530,39.664890,1,2008/2009,18,2008,2009,tz_55218021_1
5939,tz,tz_55218021,-5.367533,39.664894,2,2010/2011,16,2010,2011,tz_55218021_2
5940,tz,tz_55218021,-5.367533,39.664895,3,2012/2013,8,2012,2013,tz_55218021_3
5941,tz,tz_55218021,-5.367530,39.664890,4,2014/2015,14,2014,2015,tz_55218021_4


In [7]:
print("Number of EAs in the sample:", len(np.unique(geos['clusterid'])))
print("Number of unique year-EA pairs:",len(np.unique(geos['cluster_year_id'])))

Number of EAs in the sample: 1794
Number of unique year-EA pairs: 5943


In [3]:
#pd.crosstab(geos['year_end'],geos['country']).style.to_latex()

In [5]:
np.mean(geos['n_households'])

11.369846878680802

In [23]:
sum(geos[['clusterid','round']].duplicated())

0

In [29]:
print("if you consider just the simple time-series, you get (n_k-1) time differences (aka observations)")
print("in this case, I would have",sum(geos.groupby('clusterid').size()-1),'observations')


if you consider just the simple time-series, you get (n_k-1) time differences (aka observations)
in this case, I would have 4149 observations


In [40]:
print("If I consider the pairwise differences between all survey rounds within one country I get (n_k C 2) time diffs")
time_diffs = geos.groupby(['country','clusterid']).size().reset_index(name = 'occurences')
time_diffs['training_obs'] = [math.comb(i,2) for i in list(time_diffs['occurences'])]
print(time_diffs.groupby("country").sum('training_obs'))
print("Number of observations:", sum(time_diffs['training_obs']))

If I consider the pairwise differences between all survey rounds within one country I get (n_k C 2) time diffs
         occurences  training_obs
country                          
eth            1195          1094
mw              612           714
ng             1589          1893
tz             1309          1634
ug             1238          2188
Number of observations: 7523
